# Get phenotypes associated with a list of genes

In [21]:
import json
import csv

In [22]:
import requests

INDEX_PAGE = "http://idr-demo.openmicroscopy.org/webclient/?experimenter=-1"

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

In [23]:
with open('FiveExampleGenes.txt') as f:
    genes = f.read().splitlines()
genes[:5]

['PSMA1', 'PSMA2', 'PSMA5', 'PREX2', 'NOTAGENE']

In [24]:
SCREENS_PROJECTS_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/?value={value}"
PLATES_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/plates/?value={value}&id={screen_id}"
IMAGES_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/images/?value={value}&node={parent_type}&id={parent_id}"
ATTRIBUTES_URL = "http://idr-demo.openmicroscopy.org/webclient/api/annotations/?type=map&image={image_id}"

In [49]:
attr_type = "gene"
attr_keys = {
    "phenotype":
        ("Phenotype",
         "Phenotype Term Name",
         "Phenotype Term Accession",
         "Phenotype Term Accession URL")
}

with open('GenesWithPhenotypes.csv','w') as csvfile:
    fieldnames = [
        'Gene Symbol', 'Screen', 'Plate', 'Image',
        'Phenotype', 'Phenotype Term Name', 'Phenotype Term Accession',
        'Phenotype Term Accession URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for gene in genes:
        #f.write ('{0}'.format(gene))
        qs1 = {'key': attr_type, 'value': gene}
        url1 = SCREENS_PROJECTS_URL.format(**qs1)

        for s in session.get(url1).json()['screens']:
            screen_id = s['id']
            screen_name = s['name']
            #print (gene, s['id'], s['name'])    
            qs2 = {'key': attr_type, 'value': gene, 'screen_id': screen_id}
            url2 = PLATES_URL.format(**qs2)

            for p in session.get(url2).json()['plates']:
                plate_id = p['id'] 
                plate_name = p['name']
                qs3 = {'key': attr_type, 'value': gene, 'parent_type': 'plate', 'parent_id': plate_id}
                url3 = IMAGES_URL.format(**qs3)

                for i in session.get(url3).json()['images']:

                    image_id = i['id']
                    url4 = ATTRIBUTES_URL.format(**{'image_id': image_id})
                    for a in session.get(url4).json()['annotations']:
                        row = {}
                        for v in a['values']:
                            if str(v[0]) in attr_keys['phenotype']:
                                if str(v[0]) not in row:
                                    row[str(v[0])] = v[1]
                        if row:
                            row.update({'Gene Symbol': gene, 'Screen': screen_id,
                                        'Plate': plate_id, 'Image': image_id})
                            writer.writerow(row)
                         
